# Usage of DeepRaman


Here, part of the liquid and powder mixture dataset is taken as an example to show the process of DeepRaman.

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.linear_model import enet_path
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt 
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Layer
from scipy.sparse import spdiags,eye,csc_matrix, diags
import copy
import csv

### Import Data


In [3]:
# Database
datafile0 = u'E:/20220608/DeepMix/DeepRaman/data/database_for_Liquid_and_powder_mixture.npy'
spectrum_pure = np.load(datafile0) 
# Unknow
datafile1 =u'E:/20220608/DeepMix/DeepRaman/data/unknown_Liquid_and_powder_mixture.npy'
spectrum_mix = np.load(datafile1)     
# Component information of database
csv_reader = csv.reader(open(u'E:/20220608/DeepMix/DeepRaman/data/database_for_Liquid_and_powder_mixture.csv', encoding='utf-8'))
DBcoms = [row for row in csv_reader]   
print('Unknow：',spectrum_mix.shape)
print('Database：',DBcoms)

Unknow： (6, 881)
Database： [['Acetonitrile_75-0508_2241'], ['Ethanol_2241'], ['Methanol_2241'], ['Polyacrylamide_2241'], ['Sodium Acetate Trihydrate _2241'], ['Sodium Carbonate_2241']]


### Data process

In [4]:
spectrum_pure_sc =  copy.deepcopy(spectrum_pure)
spectrum_mix_sc = copy.deepcopy(spectrum_mix)
for i in range(spectrum_mix.shape[0]):
    spectrum_mix_sc[i,:] = spectrum_mix[i,:]/np.max(spectrum_mix[i,:])
for i in range(spectrum_pure.shape[0]):
    spectrum_pure_sc[i,:] = spectrum_pure[i,:]/np.max(spectrum_pure[i,:])


X = np.zeros((spectrum_mix_sc.shape[0]*spectrum_pure_sc.shape[0],2,881,1))

for p in range(spectrum_mix_sc.shape[0]):
    for q in range(spectrum_pure_sc.shape[0]):
        X[int(p*spectrum_pure_sc.shape[0]+q),0,:,0] = spectrum_mix_sc[p,:]
        X[int(p*spectrum_pure_sc.shape[0]+q),1,:,0] = spectrum_pure_sc[q,:]

### Reload and predict

In [5]:
re_model = tf.keras.models.load_model('E:/20220608/DeepMix/DeepRaman/model/model.h5', custom_objects=_custom_objects)
y = re_model.predict(X)


### Baseline removal

In [6]:
spectrum_pure = WhittakerSmooth_MAT(spectrum_pure, lamb=1)
spectrum_pure = airPLS_MAT(spectrum_pure, lamb=10, itermax=10)
spectrum_mix = WhittakerSmooth_MAT(spectrum_mix, lamb=1)
spectrum_mix = airPLS_MAT(spectrum_mix, lamb=10, itermax=10)

### NN-EN for ratio estimation

In [7]:
for cc in range(spectrum_mix.shape[0]):
    com=[]
    coms = []
    ra2 = []
    for ss in range(cc*spectrum_pure.shape[0],(cc+1)*spectrum_pure.shape[0]):

        if y[ss,1]>=0.5:
            com.append(ss%spectrum_pure.shape[0])


    X = spectrum_pure[com]
    coms = [DBcoms[com[h]] for h in range(len(com))]

    _, coefs_lasso, _ = enet_path(X.T, spectrum_mix[cc,:], l1_ratio=0.96,
                              positive=True, fit_intercept=False)
    ratio = coefs_lasso[:, -1]
    ratio_sc = copy.deepcopy(ratio)

    for ss2 in range(ratio.shape[0]):
        ratio_sc[ss2]=ratio[ss2]/np.sum(ratio)


    print('The',cc, 'spectra may contain:',coms)
    print('The corresponding ratio is:', ratio_sc)

The 0 spectra may contain: [['Acetonitrile_75-0508_2241'], ['Ethanol_2241'], ['Methanol_2241']]
The corresponding ratio is: [0.19772626 0.31080356 0.49147018]
The 1 spectra may contain: [['Acetonitrile_75-0508_2241'], ['Ethanol_2241'], ['Methanol_2241']]
The corresponding ratio is: [0.0785501  0.21088816 0.71056175]
The 2 spectra may contain: [['Acetonitrile_75-0508_2241'], ['Ethanol_2241'], ['Methanol_2241']]
The corresponding ratio is: [0.30807043 0.30981864 0.38211093]
The 3 spectra may contain: [['Polyacrylamide_2241'], ['Sodium Acetate Trihydrate _2241']]
The corresponding ratio is: [0.74317879 0.25682121]
The 4 spectra may contain: [['Polyacrylamide_2241'], ['Sodium Acetate Trihydrate _2241'], ['Sodium Carbonate_2241']]
The corresponding ratio is: [0.38000182 0.14882521 0.47117298]
The 5 spectra may contain: [['Polyacrylamide_2241'], ['Sodium Acetate Trihydrate _2241'], ['Sodium Carbonate_2241']]
The corresponding ratio is: [0.61626663 0.22985714 0.15387623]
